# Basic/B1 Example
Import the `Geant4` and `Geant4.SystemOfUnits` modules

In [1]:
using Geant4
using Geant4.SystemOfUnits

### Define the function that will construct the detector geometry.
The setup consists of a an envelope of box shape containing two volumes: a spherical cone and a trapezoid.

In this example we use  some common materials materials for medical applications. The envelope is made of water and the two inner volumes
are made from tissue and bone materials.
The materials are created with the help of the G4NistManager class,
which allows to build a material from the NIST database using their
names. All available materials can be found in the Geant4 User's Guide
for Application Developers, Appendix 10: Geant4 Materials Database.

In [2]:
function constructB1Detector()
    nist = G4NistManager!Instance()
  
    # Envelope parameters
    env_sizeXY = 20cm
    env_sizeZ  = 30cm
    env_mat    = FindOrBuildMaterial(nist, "G4_WATER")
  
    # Option to switch on/off checking of volumes overlaps
    checkOverlaps = true
  
    # World
    world_sizeXY = 1.2*env_sizeXY
    world_sizeZ  = 1.2*env_sizeZ
    world_mat = FindOrBuildMaterial(nist, "G4_AIR")
  
    solidworld = G4Box("World", 0.5 * world_sizeXY, 0.5 * world_sizeXY, 0.5 * world_sizeZ)
    logicworld = G4LogicalVolume(solidworld, world_mat, "World")
  
    physWorld = G4PVPlacement(C_NULL,          # no rotation
                              G4ThreeVector(),  # at (0,0,0)
                              logicworld,       # its logical volume
                              "World",          # its name
                              C_NULL,           # its mother volume
                              false,            # no boolean operation
                              0,                # copy number
                              checkOverlaps)    # overlaps checking
  
    # Envelope
    solidEnv = G4Box("Envelope", 0.5 * env_sizeXY, 0.5 * env_sizeXY, 0.5 * env_sizeZ)
    logicEnv = G4LogicalVolume(solidEnv, env_mat, "Envelope")
  
    G4PVPlacement(C_NULL,            # no rotation
                  G4ThreeVector(),   # at (0,0,0)
                  logicEnv,          # its logical volume
                  "Envelope",        # its name
                  logicworld,        # its mother  volume
                  false,             # no boolean operation
                  0,                 # copy number
                  checkOverlaps)     # overlaps checking
    # shape 1
    shape1_mat = FindOrBuildMaterial(nist, "G4_A-150_TISSUE")
    pos1 = G4ThreeVector(0, 2cm, -7cm)
    
    # Conical section shape
    shape1_rmina =  0cm; shape1_rmaxa = 2cm
    shape1_rminb =  0cm; shape1_rmaxb = 4cm
    shape1_hz = 3cm
    shape1_phimin = 0deg; shape1_phimax = 360deg
  
    solidShape1 = G4Cons("Shape1", shape1_rmina, shape1_rmaxa, shape1_rminb, shape1_rmaxb,
                  shape1_hz, shape1_phimin, shape1_phimax)
    
    logicShape1 = G4LogicalVolume(solidShape1, shape1_mat, "Shape1")
    G4PVPlacement(C_NULL,       # no rotation
                  pos1,         # at position
                  logicShape1,  # its logical volume
                  "Shape1",     # its name
                  logicEnv,     # its mother  volume
                  false,        # no boolean operation
                  0,            # copy number
                  checkOverlaps)# overlaps checking
    # Shape 2
    shape2_mat = FindOrBuildMaterial(nist, "G4_BONE_COMPACT_ICRU")
    pos2 = G4ThreeVector(0, -1cm, 7cm)
                  
    # Trapezoid shape
    shape2_dxa = 12cm; shape2_dxb = 12cm
    shape2_dya = 10cm; shape2_dyb = 16cm
    shape2_dz  = 6cm
    solidShape2 = G4Trd("Shape2", 0.5 * shape2_dxa, 0.5 * shape2_dxb, 0.5 * shape2_dya, 0.5 * shape2_dyb, 0.5 * shape2_dz)
    logicShape2 = G4LogicalVolume(solidShape2, shape2_mat, "Shape2")
    G4PVPlacement(C_NULL,         # no rotation
                  pos2,           # at position
                  logicShape2,    # its logical volume
                  "Shape2",       # its name
                  logicEnv,       # its mother  volume
                  false,          # no boolean operation
                  0,              # copy number
                  checkOverlaps)  # overlaps checking
  
    return physWorld              # return a pointer to the G4PhysicalVolume
  end  

constructB1Detector

### Detector constructor object and physics list 

In [3]:
detdesc = G4JLDetectorConstruction(constructB1Detector)
physics = QBBC()
#SetVerboseLevel(physics, 1)

<<< Reference Physics List QBBC 


Geant4.QBBCAllocated(Ptr{Nothing} @0x00007f8f39e9f230)

### User Actions

In [4]:
# User Actions
function buildApp(self::G4JLActionInitialization)
    # Create and register particle gun
    particle_gun = G4JLParticleGun()
    SetUserAction(self, particle_gun)
  
    # Setup particle gun
    pg = GetGun(particle_gun)
    SetParticleByName(pg, "e-")
    SetParticleEnergy(pg, 100MeV)
    SetParticleMomentumDirection(pg, G4ThreeVector(0,0,1))
    SetParticlePosition(pg, G4ThreeVector(0,0,-16cm))
  end
  app = G4JLActionInitialization(buildApp)

CxxWrap.CxxWrapCore.CxxPtr{G4JLActionInitialization}(Ptr{G4JLActionInitialization} @0x00007f8f3489e0b0)

### Instantiate RunManager and initialize

In [5]:
# Construct the default run manager and set initializers
runManager = G4RunManager()

SetUserInitialization(runManager, detdesc)
SetUserInitialization(runManager, ptr(physics))
SetUserInitialization(runManager, app)

# Get the pointer to the User Interface manager
UImanager = G4UImanager!GetUIpointer();


**************************************************************
 Geant4 version Name: geant4-11-01 [MT]   (9-December-2022)
                       Copyright : Geant4 Collaboration
                      References : NIM A 506 (2003), 250-303
                                 : IEEE-TNS 53 (2006), 270-278
                                 : NIM A 835 (2016), 186-225
                             WWW : http://geant4.org/
**************************************************************



### Execute the commands to initiliate and run few events

In [6]:
# Initialize kernel
ApplyCommand(UImanager, "/run/initialize")

# Change verbosity
#ApplyCommand(UImanager, "/control/verbose 2")
#ApplyCommand(UImanager, "/run/verbose 2")
#ApplyCommand(UImanager, "/event/verbose 0")
ApplyCommand(UImanager, "/tracking/verbose 1")

# gamma 6 MeV to the direction (0.,0.,1.)
ApplyCommand(UImanager, "/gun/particle gamma")
ApplyCommand(UImanager, "/gun/energy 6 MeV")

# go
ApplyCommand(UImanager, "/run/beamOn 1")

Checking overlaps for volume Envelope:0 (G4Box) ... OK! 
Checking overlaps for volume Shape1:0 (G4Cons) ... OK! 
Checking overlaps for volume Shape2:0 (G4Trd) ... OK! 
### HadronInelasticQBBC Construct Process:
    Emin(FTFP)= 3 GeV; Emax(FTFP)= 100000 GeV
    Emin(BERT)= 1 GeV; Emax(BERT)= 6 GeV; Emax(BERTpions)= 12 GeV;
    Emin(BIC) = 0 GeV; Emax(BIC)= 1.5 GeV.
======                 Electromagnetic Physics Parameters      ========
LPM effect enabled                                 1
Enable creation and use of sampling tables         0
Apply cuts on all EM processes                     0
Use combined TransportationWithMsc                 Disabled
Use general process                                1
Enable linear polarisation for gamma               0
Enable photoeffect sampling below K-shell          1
Enable sampling of quantum entanglement            0
X-section factor for integral approach             0.8
Min kinetic energy for tables                      100 eV 
Max kinetic ener

0

In [8]:
ApplyCommand(UImanager, "/gun/particle e+")
ApplyCommand(UImanager, "/gun/energy 60 MeV")
ApplyCommand(UImanager, "/run/beamOn 1")


*********************************************************************************************************
* G4Track Information:   Particle = e+,   Track ID = 1,   Parent ID = 0
*********************************************************************************************************

Step#    X(mm)    Y(mm)    Z(mm) KinE(MeV)  dE(MeV) StepLeng TrackLeng  NextVolume ProcName
    0        0        0     -160        60        0        0         0       World initStep
    1        0        0     -150        60  0.00326       10        10    Envelope Transportation
    2   -0.866    -1.73     -102      50.5     8.25     48.2      58.2    Envelope eIoni
    3   -0.831    -1.86     -100      49.7    0.273      1.8        60    Envelope eIoni
    4   -0.616    -2.21    -93.3      48.5     1.18      6.8      66.8      Shape1 Transportation
    5   -0.549    -2.34    -92.1      47.5    0.229     1.29      68.1      Shape1 eIoni
    6   -0.529    -2.41    -91.5      46.9    0.207    0.576      6

0